In [21]:
import pickle
import pandas as pd
import numpy as np

%cd ~/openfold
# %rm out/experiments/angletransformer-make-caches-00/*.pkl
%ls -hlt out/experiments/angletransformer-make-caches-01230723/


/net/pulsar/home/koes/jok120/openfold
total 62M
-rw-r--r--  1 jok120 koes  16M Jul 31 12:29 debug_angle_transformer_intermediates1_train.pkl
-rw-r--r--  1 jok120 koes  16M Jul 31 12:29 debug_angle_transformer_intermediates0_train.pkl
-rw-r--r--  1 jok120 koes  16M Jul 31 12:29 debug_angle_transformer_intermediates2_train.pkl
-rw-r--r--  1 jok120 koes  16M Jul 31 12:29 debug_angle_transformer_intermediates3_train.pkl
drwxr-xr-x  4 jok120 koes   48 Jul 31 12:28 finetune-openfold-02/
drwxr-xr-x 18 jok120 koes 4.0K Jul 31 12:27 wandb/
drwxr-xr-x  4 jok120 koes   40 Jul 31 12:20 pdbs/


In [28]:
idx = 3
fn = f"out/experiments/angletransformer-make-caches-01230723/debug_angle_transformer_intermediates{idx}_train.pkl"

with open(fn, "rb") as f:
    intermediates = pickle.load(f)

In [29]:
intermediates.keys()

dict_keys(['AT', 'batch'])

In [30]:
d = intermediates

In [34]:
d['AT']['s']

[tensor([[[ 0.0625,  0.1226,  0.4668,  ...,  0.0679,  0.1592, -0.0096],
          [ 0.0032,  0.0864,  0.4082,  ..., -0.0317,  0.1484,  0.0688],
          [ 0.1196,  0.0801,  0.3340,  ..., -0.0073,  0.1396, -0.0094],
          ...,
          [-0.1162,  0.0728,  0.1177,  ..., -0.0262,  0.0620, -0.0361],
          [ 0.0364, -0.0128,  0.1426,  ..., -0.0168,  0.0684,  0.0255],
          [-0.2109,  0.0549,  0.0223,  ...,  0.0352,  0.0474, -0.0464]]],
        device='cuda:3', dtype=torch.bfloat16),
 tensor([[[ 0.0056,  0.0132, -0.0156,  ...,  0.0142,  0.0166, -0.0031],
          [ 0.0025,  0.0231,  0.0045,  ..., -0.0192,  0.0242, -0.0011],
          [ 0.0260,  0.0021,  0.0177,  ..., -0.0002,  0.0146,  0.0024],
          ...,
          [-0.1318,  0.1299,  0.0923,  ..., -0.0386,  0.0354,  0.0277],
          [-0.1030,  0.1143,  0.0476,  ...,  0.0364,  0.0408, -0.0476],
          [-0.1777,  0.0488,  0.0771,  ..., -0.0398,  0.0312,  0.0152]]],
        device='cuda:3', dtype=torch.bfloat16),
 tenso

In [32]:
d['batch']['name']

[['1kb5_H'], ['3wlw_A'], ['1zps_A'], ['2ypl_E'], ['6ujs_A'], ['5f9a_A']]

In [27]:
d['batch']['name']

[['2l5a_A'], ['6k8h_A'], ['5sxg_A'], ['4ny4_A'], ['4xbt_A'], ['2rjl_A']]

In [109]:
len(d['inputs']), len(d['outputs']), len(d['loss_inputs']), len(d['loss_outputs'])

KeyError: 'inputs'

In [110]:
d["batch"]

{'aatype': [], 'seq_mask': [], 'chi_mask': [], 'chi_angles_sin_cos': []}

In [102]:
d['loss_outputs'][0][0]

KeyError: 'loss_outputs'

In [103]:
d['loss_inputs'][0][1]

KeyError: 'loss_inputs'

### Things I need based on the below:

For supervised_chi_loss:
* Input
  - AT input, aka s and s_initial 
      - (1 x 256 x 384), (1 x 256 x 384), d['inputs'][0]
* Output
    * batch["aatype"]
    * batch["seq_mask"]
    * batch["chi_mask"]
    * batch["chi_angles_sin_cos"] (GT angles)

 

In [94]:
import torch
# AT input: s, s_initial (1 x 256 x 384), (1 x 256 x 384)
print(list(map(lambda x: x.shape, d['inputs'][0])))

# AT output: unnormalized_angles, angles (1 x 256 x 7, 2), (1 x 256 x 7, 2)
print(list(map(lambda x: x.shape, d['outputs'][0])))

# Loss_input: (out, batch)
# out: dict: ['msa', 'pair', 'single', 'sm', 'final_atom_positions', 'final_atom_mask', 
# 'final_affine_tensor', 'lddt_logits', 'plddt', 'distogram_logits', 'masked_msa_logits', 
# 'experimentally_resolved_logits', 'violation']
print(d['loss_inputs'][0][0]['sm']["angles"].shape)
print(d['loss_inputs'][0][1]["chi_angles_sin_cos"].shape)

# Loss_output: (cum_loss, loss_dict)
print(d['loss_outputs'][0])





[torch.Size([1, 256, 384]), torch.Size([1, 256, 384])]
[torch.Size([1, 256, 7, 2]), torch.Size([1, 256, 7, 2])]
torch.Size([8, 1, 256, 7, 2])
torch.Size([1, 256, 4, 2])
(tensor(42.3516, device='cuda:2', requires_grad=True), {'distogram': tensor(0.2772, device='cuda:2'), 'experimentally_resolved': tensor(0.0010, device='cuda:2'), 'fape': tensor(0.4888, device='cuda:2'), 'plddt_loss': tensor(0.0163, device='cuda:2'), 'masked_msa': tensor(1.4615, device='cuda:2'), 'sq_chi_loss': tensor([1.4747], device='cuda:2', requires_grad=True), 'angle_norm_loss': tensor([2.4033], device='cuda:2', requires_grad=True), 'angle_mae': tensor(1.4212, device='cuda:2', requires_grad=True), 'supervised_chi': tensor(0.7614, device='cuda:2'), 'violation': tensor(0.2046, device='cuda:2'), 'openmm_unscaled': tensor(1., device='cuda:2'), 'openmm_scaled': tensor(1.0000e-06, device='cuda:2'), 'openmm_raw_energy': tensor(7.5165e+12, device='cuda:2'), 'openmm': tensor(0.), 'unscaled_loss': tensor(3.2107, device='cuda:

In [87]:
d['loss_inputs'][0][0].keys()

dict_keys(['msa', 'pair', 'single', 'sm', 'final_atom_positions', 'final_atom_mask', 'final_affine_tensor', 'lddt_logits', 'plddt', 'distogram_logits', 'masked_msa_logits', 'experimentally_resolved_logits', 'violation'])

In [ ]:
# Make a simple resnet with two layers

model = nn.Sequential(
    nn.Linear(2, 2),
    nn.ReLU(),
    nn.ReLU(),
    nn.Linear(2, 2),
    nn.ReLU(),
    nn.Linear(2, 2),
    nn.ReLU(),
    nn.Linear(2, 2),
    nn.ReLU(),
    nn.Linear(2, 2),

)
